In [ ]:
pip install scikit-learn geopandas h3pandas h3~=3.0 -q

In [2]:
import geopandas as gpd
import h3pandas
from shapely.geometry import Point, Polygon
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [3]:
schools = gpd.read_file("data/nyc/SchoolPoints_APS_2024_08_28 (1)/SchoolPoints_APS_2024_08_28.shp")
subways = gpd.read_file("data/nyc/nyc_subway_entrances/nyc_subway_entrances.shp")
bike_paths = gpd.read_file("data/nyc/New York City Bike Routes_20241223.geojson")
neighborhoods = gpd.read_file("https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson")
parks = gpd.read_file("data/nyc/Parks Properties_20241223.geojson")

In [4]:
schools = schools.to_crs("EPSG:3857")
subways = subways.to_crs("EPSG:3857")
bike_paths = bike_paths.to_crs("EPSG:3857")
neighborhoods = neighborhoods.to_crs("EPSG:3857")
parks = parks.to_crs("EPSG:3857")

# Analyze neighborhoods

In [107]:
# Analysis for each neighborhood
def analyze_neighborhood(neighborhood_geometry):
    # Count features intersecting the neighborhood boundary
    num_schools = schools[schools.geometry.intersects(neighborhood_geometry)].shape[0]
    num_subways = subways[subways.geometry.intersects(neighborhood_geometry)].shape[0]
    bike_path_length = bike_paths[bike_paths.geometry.intersects(neighborhood_geometry)].length.sum()
    park_area = parks[parks.geometry.intersects(neighborhood_geometry)].area.sum()

    return num_schools, num_subways, bike_path_length, park_area

In [108]:
# Apply analysis to each neighborhood
neighborhoods[['num_schools', 'num_subways', 'bike_path_length', 'park_area']] = neighborhoods.geometry.apply(
    lambda geom: pd.Series(analyze_neighborhood(geom))
)

In [ ]:
neighborhoods.head(3)

In [ ]:
neighborhoods.head(3)

In [111]:
# Normalize results (0 to 1 scale)
scaler = MinMaxScaler()
columns_to_normalize = ['num_schools', 'num_subways', 'bike_path_length', 'park_area']
neighborhoods[columns_to_normalize] = scaler.fit_transform(neighborhoods[columns_to_normalize])

In [112]:
# Aggregate results using touching neighborhoods
def aggregate_touching_neighborhoods(neighborhood_index):
    current_geometry = neighborhoods.loc[neighborhood_index, 'geometry']
    touching_indices = neighborhoods[neighborhoods.geometry.touches(current_geometry)].index

    if not touching_indices.empty:
        neighbor_values = neighborhoods.loc[touching_indices, columns_to_normalize].mean()
    else:
        neighbor_values = neighborhoods.loc[neighborhood_index, columns_to_normalize]

    return neighbor_values

In [113]:
# Apply aggregation to each neighborhood
neighborhoods[columns_to_normalize] = neighborhoods.index.to_series().apply(
    lambda idx: aggregate_touching_neighborhoods(idx)
)


In [99]:
# Final normalization (0 to 1 scale)
neighborhoods[columns_to_normalize] = scaler.fit_transform(neighborhoods[columns_to_normalize])

In [ ]:
neighborhoods.head(3)

In [115]:
neighborhoods['index_score'] = neighborhoods['num_schools'] + neighborhoods['num_subways'] + neighborhoods['bike_path_length'] + neighborhoods['park_area'] 

In [ ]:
import leafmap

m = leafmap.Map()
m.add_data(
    neighborhoods, column="index_score", scheme="Quantiles", cmap="Blues", legend_title="Index"
)
m

In [117]:
# Save or visualize the results
neighborhoods.to_file("neighborhood_access_index.geojson", driver="GeoJSON")

# Set up the H3 Grid

In [29]:
neighborhoods = neighborhoods.to_crs('EPSG:4326')

In [30]:
resolution = 9  # Adjust resolution as needed
gdf_h3 = neighborhoods.h3.polyfill(resolution)

In [ ]:
gdf_h3.head()

In [ ]:
gdf_h3 = neighborhoods.h3.polyfill(resolution, explode=True)
gdf_h3.head()

In [ ]:
gdf_h3 = gdf_h3[gdf_h3['h3_polyfill'].isnull() == False].set_index('h3_polyfill')
gdf_h3.index.name = None
gdf_h3

In [34]:
gdf_h3 = gdf_h3.h3.h3_to_geo_boundary()

In [ ]:
pip install folium matplotlib mapclassify -q

In [ ]:
gdf_h3.explore()

In [37]:
gdf_h3_proj = gdf_h3.to_crs('EPSG:3857')

In [38]:
# Analysis for each hex cell
def analyze_access(hex_geometry):
    # Buffer hex geometry
    buffer_1600m = hex_geometry.buffer(1600)
    buffer_800m = hex_geometry.buffer(800)

    # Count features within buffers
    num_schools = schools[schools.geometry.intersects(buffer_1600m)].shape[0]
    num_subways = subways[subways.geometry.intersects(buffer_1600m)].shape[0]
    bike_path_length = bike_paths[bike_paths.geometry.intersects(buffer_1600m)].length.sum()
    park_area = parks[parks.geometry.intersects(buffer_800m)].area.sum()

    return num_schools, num_subways, bike_path_length, park_area

In [39]:
gdf_h3_proj = gdf_h3.to_crs('EPSG:3857')

In [40]:
gdf_h3_proj[['num_schools', 'num_subways', 'bike_path_length', 'park_area']] = gdf_h3_proj.geometry.apply(
    lambda hex_geom: pd.Series(analyze_access(hex_geom))
)


In [ ]:
gdf_h3_proj.head()

In [42]:
gdf_h3_proj['h3_index'] = gdf_h3_proj.index

# Run the normalization analysis

In [43]:
import h3

In [50]:
# Normalize results
scaler = MinMaxScaler()
normalized_columns = ['num_schools', 'num_subways', 'bike_path_length', 'park_area']
gdf_h3_proj[normalized_columns] = scaler.fit_transform(gdf_h3_proj[normalized_columns])

# Aggregate results using neighboring cells
def aggregate_neighbors(h3_index):
    neighbors = h3.k_ring(h3_index, 2)  # 2-k ring
    neighbor_values = gdf_h3_proj[gdf_h3_proj['h3_index'].isin(neighbors)][normalized_columns].mean()
    return neighbor_values

gdf_h3_proj[normalized_columns] = gdf_h3_proj['h3_index'].apply(
    lambda h3_index: aggregate_neighbors(h3_index)
)

# # Final normalized analysis
gdf_h3_proj[normalized_columns] = scaler.fit_transform(gdf_h3_proj[normalized_columns])

# Save or visualize the results
gdf_h3_proj.to_file("access_index.geojson", driver="GeoJSON")

In [ ]:
gdf_h3_proj.head(3)

# Create the total score

In [52]:
gdf_h3_proj['index_score'] = gdf_h3_proj['num_schools'] + gdf_h3_proj['num_subways'] + gdf_h3_proj['bike_path_length'] + gdf_h3_proj['park_area'] 

In [53]:
import leafmap

In [54]:
gdf_h3_map = gdf_h3_proj.to_crs('EPSG:4326')

In [ ]:
m = leafmap.Map()
m.add_data(
    gdf_h3_map, column="index_score", scheme="Quantiles", cmap="Blues", legend_title="Index"
)
m